# Google's Data Analytics Capstone Project - Track 1

## Introduction

In this project, we'll act as a junior data analyst working on the marketing analyst team at Bellabeat, a high-tech manufacturer of health-focused
products for women. Where Urška Sršen, cofounder and Chief Creative Officer of Bellabeat, believes that analyzing smart
device fitness data could help unlock new growth opportunities for the company. We have been asked to analyze smart device data to gain insights into how customers are using their devices and help guide the marketing strategy for Bellabeat.

For this project, we'll refer from using Excel or sheets since the objective is to showcase SQL, R and Tableau (also because we don't have an Excel license on the PC we're working with). We'll limit to opening files with Excel to peek at the info and the structure.

## Ask

We'll begin by defining our business task:

`Identify trends in smart device usage and use these trends to influence Bellabeat marketing strategy`

Which we can then break down into the following questions:
* What are some trends in smart device usage?
* How could these trends apply to Bellabeat customers?
* How could these trends help influence Bellabeat marketing strategy?

## Prepare

We are referred to the following dataset:
 <a href="https://www.kaggle.com/datasets/arashnic/fitbit" target="_blank">Fitbit fitness tracker</a>(Public domain, available through Kaggle) 
Which contains data from fitbit users, including information for physical activity, heart rate, weight and sleep monitoring. This information is divided in 18 CSV documents which contain the information at different levels such as daily, hourly or by minute and also it is arranged in wide and long format.

We begin importing these documents in BigQuery and after some exploratory analysis we encounter certain limitations with this dataset:
* Neither gender nor age are disclosed for users
* With only 33 users and data for 1 month the sample size is too small to make valid assumptions
* Not all segments (sleep, activity, weight) have the same number of users contributing their data
* Some datasets do not include information for the units they're measuring with (columns such as: distance, instensity,fat)

![Total users in daily activity](img/users_daily_activity.png)

Total users in activity data

![Total users in sleep activity](img/users_sleep_activity.png)

Total users in sleep data

To address these limitations we can search for more complete datasets or work under some assumptions. While looking for other datasets we come across the <a href="https://zenodo.org/record/53894#.X9oeh3Uzaao" target="_blank">original source</a> for this data which contains an additional month of information. We also note that the researchers used a company called Fitabase to source this data, upon looking at their website we come across a <a href="https://www.fitabase.com/media/1930/fitabasedatadictionary102320.pdf" target="_blank">data dictionary</a> that will be helpful in clarifying some questions we had about the data such as what units correspond to some columns in different datasets.

### Considerations

As the second dataset only contains data in long format and anyway it's easier to work with this format we'll ignore the wide format files, also in the original 18 CSV files there are 3 files which contain daily data for steps, intensities and calories which we'll also ignore since this information is summarized in the daily activity file. Another file we won't be using for this analysis is the one for sleep records by minute, which contains information about the quality of sleep at night for the users, instead we'll use the SleepDay file that contains the amount of time a person is sleeping by day, though we have to note again that this file is not present in the 2nd dataset so we'll only analize 1 month for this data.


We will be appending the 2 datasets where applicable and also will group the data by time section (we'll explain this in the next phase of the project) therefore the raw data that we will be working with is the following:

|Data file|Summary|Time Period|
|---|---|---|
|Daily Activity  |Daily steps, distance(km), calories and intensity by distance/minutes|2016-03-12 - 2016-05-12|
|Heart Rate Activity|Heart rate values by the minute|2016-03-12 - 2016-05-12|
|Hourly Activity|Hourly steps, calories and total/average intensities |2016-03-12 - 2016-05-12|
|METs|Metabolic equivalents by the minute|2016-03-12 - 2016-05-12|
|Weight Logs|Weight (kg/lbs), fat, BMI and boolean for whether it was a manual report|2016-03-30 - 2016-05-12|
|Sleep Activity|Daily total times asleep, minutes asleep and minutes in bed|2016-04-12 - 2016-05-12|

## Process

### Parsing dates

When we first open the CSV files contained in the dataset we identify the date variables as a timestamp, however when trying to upload them to BigQuery most of them can't be parsed as they are. This is because, according to the source, there are differences in format between devices and personal preferences too. To work around this, we can import them as strings, parse and separate them in another SQL query with a temp table and the help of the functions: `PARSE_DATETIME(format_string, datetime_string)` and `EXTRACT(part FROM datetime_expression)`.

![Parsing and extracting query for heartrate data](img/heartrate_parsed_query.png)

Even when having different formats for datetime in the same table these functions can parse them correctly. We also have to note that the discovery of the 2nd part of the dataset was made after importing most of the files from the 1st part. This isn't a problem though, since we can easily append the two datasets in each query for parsing the dates with `UNION ALL`. Also, the use of a temp table can be skipped and just generate the query with the parsing of dates, some testing is needed to define which one is faster but the latter produces a more readable query.

![Parsing and appending query for heartrate data](img/heartrate_parsed_union_query.png)

### Connecting BigQuery with R
Because some of the tables that we are dealing with have millions of rows and to ease on the computational load on our local machine we can connect to BigQuery from R and perform any query using BQ's computational power.

Let's begin by loading our packages, in this project we'll use `DBI` and `bigrquery` to connect R to BigQuery. Also, well load `tidyverse` because we'll be using `dbplyr` to _translate_ our code into a query.

In [10]:
library(DBI)
library(bigrquery)
library(tidyverse)
options( warn = -1 )

Once we have our packages loaded we have to create a connection to our database, we do this with the help of `dbConnect()` which has the following syntax
```R
con <- dbConnect( 
bigrquery::bigquery(), 
project = "PROJECT_ID",
dataset = DATASET_NAME, 
billing = PROJECT_ID )
```
Note that this will create a connection to BigQuery and handle the auth from our web browser, there are ways to handle the auth without user input but that's outside the scope of this project. Our connection is done as follows:

In [11]:
con <- dbConnect(
bigrquery::bigquery(),
project = "warm-utility-374917",
dataset = "Bellabeat"
)

There are other useful functions like `dbListTables()` to see what tables are present in our dataset and `bigrquery::bq_auth()` which can be used to handle our auth or, for example, to change users.

### Creating queries
The way bigrquery handles queries is lazy, what this means is that the info is not stored locally and will not be stored until we use `collect()`. Also these queries are stored as a table element, we can see this in the class of any table we create with `tbl()`. This is useful because we can use dplyr verbs in the same way we would with a tibble or a dataframe and the package will translate it automatically to a query.


Let's see this in action with one our tables, the heartrate activity table. We will begin by making a pointer to the table with `tbl()`, however we encounter the following error:
>In as.integer.integer64(x) : NAs produced by integer overflow
Which indicates that sinceour IDs are stored as integers the whole number cannot be displayed, we'll change this by using `mutate()` and `as.numeric()` to transform it to a numeric format. We continue building the query introducing a `distinct()` clause to drop possible duplicates, finally we include another `mutate()` to create a datetime column. 

In [12]:
heartrate_activity <- tbl(con, "heartrate_activity_merged") %>%
    mutate(Id=as.numeric(Id)) %>%
    distinct() %>%
    mutate(datetime=sql("DATETIME(date,time)"))    
class(heartrate_activity)

[1] "tbl_BigQueryConnection" "tbl_dbi"                "tbl_sql"               
[4] "tbl_lazy"               "tbl"

We can check the query generated by our previous code by including the function `show_query()`. Which gives us the following query:
``` SQL
SELECT *, DATETIME(date,time) AS `datetime`
FROM (
  SELECT DISTINCT SAFE_CAST(`Id` AS FLOAT64) AS `Id`, `date`, `time`, `Value`
  FROM `heartrate_activity_merged`
) `q01`
```
We now have out heartrate data in a table we can query with all the info correctly formatted. However there are other issues we need to address. First, this data is measured in an interval of 30-60 seconds which gives us a table of 3.4+ million rows. This is the reason why we are working with BigQuery from R, saving this locally would probably not be very fast depending on the specs of our local machine. Second, even if BQ has no trouble working with this volume of data there is a bill every time a query is performed. We have 1TB of processing quota every month and even if this query is ~113 MB everytime we query the table, we can optimize this by grouping our data by the hour.

We can address this by creating a new table and querying from that table onwards. We'll use `as_data_frame()` to transform our table to a dataframe that can be then uploaded to BQ with `dbWriteTable()`. 

In [13]:
heartrate_activity %>%
    mutate(hour=sql("DATETIME_TRUNC(datetime, hour)")) %>%
    select(Id,hour,Value) %>%
    group_by(Id,hour ) %>%
    summarise(avg_heartrate=mean(Value),min_heartrate=min(Value),max_heartrate=max(Value),num_readings=count(Value)) #%>%
    as_data_frame() %>%
    dbWriteTable(con,"heartrate_activity_hourly",.)

`summarise()` has grouped output by "Id". You can override using the `.groups`
argument.


# Source:   SQL [?? x 6]
# Database: BigQueryConnection
# Groups:   Id
           Id hour                avg_heartrate min_heartrate max_heartrate
        <dbl> <dttm>                      <dbl>         <int>         <int>
 1 6391747486 2016-04-05 21:00:00          93.5            70           123
 2 6391747486 2016-04-05 22:00:00          93.7            83           133
 3 6391747486 2016-04-08 07:00:00          60.9            49            77
 4 6391747486 2016-04-08 05:00:00          64.3            54           114
 5 2026352035 2016-04-02 00:00:00          62.3            57            68
 6 2026352035 2016-04-02 01:00:00          70.8            60            80
 7 6391747486 2016-04-05 15:00:00          84.7            64           102
 8 6391747486 2016-04-05 17:00:00          80.6            68           107
 9 2026352035 2016-04-02 02:00:00          69.7            64            75
10 6391747486 2016-04-06 05:00:00          70              70            70
# ℹ more rows
# ℹ

ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'value' in selecting a method for function 'dbWriteTable': el argumento "x" está ausente, sin valor por omisión


With the "heartrate_activity_hourly" table now in our BQ dataset we reduce the processed data from ~113 MB to ~10 MB and the number of rows from 3.4 million to 8499.

We can then create another table that references our summarized information

In [ ]:
heartrate_hourly_activity <- tbl(con,"heartrate_activity_hourly")

### Joining hourly data
Since we have data for calories, steps and intensities grouped by the same time interval of one hour, we can join these three tables into one to explore it in a single source. 
We'll do this by first creating pointers to all three of our tables, we will make a datetime column in order for the join to be simpler although dplyr can handle the join on those three different columns. 

In [ ]:
calories_hourly <- tbl(con, "calories_merged") %>%
    transmute(Id,Calories, Datetime=sql("DATETIME(Date,Time)"))
steps_hourly <- tbl(con,"steps_merged") %>%
    transmute(Id,StepTotal, Datetime=sql("DATETIME(Date,Time)"))
intensity_hourly <- tbl(con,"intensities_merged") %>%
    transmute(Id,TotalIntensity,AverageIntensity, Datetime=sql("DATETIME(Date,Time)"))

Next, let's join our three tables into a single one and upload it to our BQ project

In [14]:
hourly_data <- inner_join(calories_hourly,steps_hourly, by=c("Id","Datetime")) %>% 
    inner_join(intensity_hourly, by=c("Id","Datetime")) %>%
    mutate(Id=as.numeric(Id)) %>%
    relocate(Datetime,.after=Id) %>%
    arrange(Datetime) %>%
    as.data.frame() %>%
    dbWriteTable(con,"data_hourly",.)

ERROR: [1m[33mError[39m in `signal_reason()`:[22m
[33m![39m Already Exists: Table warm-utility-374917:Bellabeat.data_hourly [duplicate] 
